In [1]:
import snowflake.connector as sf

ctx = sf.connect(
    user='mgupta1@chewy.com', # chewy email address
    autocommit = 0,
    authenticator= "externalbrowser",
    role= "MRCH_USER_MARGIN",
    database= "EDLDB",
    warehouse= "MRCH_XL_WH",
    account='chewy.us-east-1'
    )
cs = ctx.cursor()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://chewy.okta.com/app/snowflake/exkmzig18yW58r3j00x7/sso/saml?SAMLRequest=lZJfb9owFMW%2FSuQ9J3YoFLCAisEQSKygJqyibyYxYHDs1NchYZ9%2BDn%2Bm7qGV9hY55%2Fh3fM%2FtPVWZ9E7cgNCqj8KAII%2BrRKdC7fpoFU%2F8DvLAMpUyqRXvozMH9DToActkToeF3asX%2Fl5wsJ67SAGtf%2FRRYR

In [2]:
import pandas as pd

In [3]:
merged_data = pd.read_csv("new_cohort.csv")
# model2_results = pd.read_csv("model2_result.csv")
# model3_results = pd.read_csv("model3_result.csv")

# combined_results = model1_results.merge(model2_results , on = 'CUSTOMER_ID' , how = 'left').merge(model3_results , on = 'CUSTOMER_ID' , how = 'left')
# for cols in ["PROMOTION_REQUIRED_FLAG" , "REACTIVATION_FLAG"] :
#     combined_results[cols] = combined_results[cols].astype(bool)

In [4]:
merged_data.columns

Index(['Unnamed: 0', 'BUCKETS', 'REGULAR_FLAG', 'CONSISTENT_FLAG',
       'CUSTOMER_ID', 'Flag'],
      dtype='object')

In [5]:
merged_data.shape

(7429352, 6)

In [6]:


database = 'EDLDB'
schema = 'MRCH_SANDBOX'
table_name =  'MRCH_DS_LOYALTY_CUST_SEGMENT_META_COHORT'  
# table_name =  'MRCH_DS_LOYALTY_CUST_SEGMENT_2_3'  

In [7]:
merged_data = merged_data[['REGULAR_FLAG', 'CONSISTENT_FLAG', 'BUCKETS', 'CUSTOMER_ID', "Flag" ]]

In [9]:
merged_data.dtypes

REGULAR_FLAG        int64
CONSISTENT_FLAG     int64
BUCKETS            object
CUSTOMER_ID         int64
Flag               object
dtype: object

In [10]:
merged_data.loc[merged_data["REGULAR_CUSTOMER"] == 1 , 'REGULAR_CUSTOMER'] = "REGULAR CUSTOMER"
merged_data.loc[merged_data["REGULAR_CUSTOMER"] == 0 , 'REGULAR_CUSTOMER'] = "NON - REGULAR CUSTOMER"

# merged_data.loc[merged_data["Flag"] == 'Test' , 'Flag'] = "WAVE1_TEST"
# merged_data.loc[merged_data["Flag"] == 'Control' , 'Flag'] = "WAVE1_CONTROL"

merged_data.rename(columns = {"REGULAR_CUSTOMER" : "TYPE"  } , inplace = True)


/var/folders/sk/kxj243qx5nb7mdp5tl3ldj540000gq/T/ipykernel_82033/1325596419.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'REGULAR CUSTOMER' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  merged_data.loc[merged_data["REGULAR_CUSTOMER"] == 1 , 'REGULAR_CUSTOMER'] = "REGULAR CUSTOMER"


In [11]:
merged_data

,TYPE,COHORT,QUARTILE,CUSTOMER_ID,Flag,WAVE
0,NON - REGULAR CUSTOMER,COHORT_1,4,38529467,Test,2
1,NON - REGULAR CUSTOMER,COHORT_1,1,58484991,Control,2
2,NON - REGULAR CUSTOMER,COHORT_1,3,198609663,Test,2
3,NON - REGULAR CUSTOMER,COHORT_1,1,102819816,Test,2
4,REGULAR CUSTOMER,COHORT_2,1,258569276,Control,2
...,...,...,...,...,...,...
281326,REGULAR CUSTOMER,COHORT_1,2,5391134,Control,3
281327,NON - REGULAR CUSTOMER,COHORT_1,3,112998987,Control,3
281328,NON - REGULAR CUSTOMER,COHORT_1,2,91739723,Test,3
281329,REGULAR CUSTOMER,COHORT_1,2,17083237,Test,3


In [10]:
# stringColumns=['TYPE' , "COHORT" , "Flag"]
# integerColumns=['CUSTOMER_ID' , 'QUARTILE' , 'WAVE']
# floatColumns=['REACTIVATION_PROPENSITY_SCORE' , "REACTIVATION_THRESHOLD" , "PROMOTION_REQUIRED_PROPENSITY_SCORE" , "PROMOTION_REQUIRED_THRESHOLD" ]
# dateTimeColumns=['SNAPSHOT_DATE']

integerColumns=['REGULAR_FLAG' , "CONSISTENT_FLAG" , "CUSTOMER_ID"]
stringColumns=['BUCKETS', 'Flag']

In [11]:
for currentStringColumn in stringColumns:
    merged_data[currentStringColumn]=merged_data[currentStringColumn].astype(str)
    merged_data[currentStringColumn]="'"+merged_data[currentStringColumn]+"'"
for currentIntegerColumn in integerColumns:
    merged_data[currentIntegerColumn]=merged_data[currentIntegerColumn].astype(int)
# for currentFloatColumn in floatColumns:
#     merged_data[currentFloatColumn]=merged_data[currentFloatColumn].astype(float)

In [12]:
columnsList=list(['REGULAR_FLAG', 'CONSISTENT_FLAG', 'BUCKETS', 'CUSTOMER_ID', "Flag"])

## Use the below code to create a new table

In [13]:
create_tbl_statement = "CREATE OR REPLACE TABLE " + database + "." + schema + "." + table_name + " ("

# Loop through each column finding the datatype and adding it to the statement

for column in columnsList:
    if (merged_data[column].dtype.name == "int" or merged_data[column].dtype.name == "int64"):
        create_tbl_statement = create_tbl_statement + column + " int"
    elif merged_data[column].dtype.name == "datetime64[ns]":
        create_tbl_statement = create_tbl_statement + column + " datetime"
    elif merged_data[column].dtype.name == "float64":
        create_tbl_statement = create_tbl_statement + column + " float8"
    elif merged_data[column].dtype.name == "bool":
        create_tbl_statement = create_tbl_statement + column + " boolean"
    else:
        create_tbl_statement = create_tbl_statement + column + " varchar(16777216)"

    if merged_data[column].name != merged_data.columns[-1]:
        create_tbl_statement = create_tbl_statement + ", "
    else:
        create_tbl_statement = create_tbl_statement + ")"
create_tbl_statement

'CREATE OR REPLACE TABLE EDLDB.MRCH_SANDBOX.MRCH_DS_LOYALTY_CUST_SEGMENT_META_COHORT (REGULAR_FLAG int, CONSISTENT_FLAG int, BUCKETS varchar(16777216), CUSTOMER_ID int, Flag varchar(16777216))'

In [14]:
ctx.cursor().execute(create_tbl_statement)

## Use below code to insert data into table

In [15]:
def formQuery(baseQuery, tempData, columnsList):
    for index, row in tempData.iterrows():
        baseQuery+= "("
        formRowQuery=""
        for currentColumn in columnsList:
            formRowQuery+= str(row[currentColumn]) + ", "
        baseQuery+=formRowQuery[:-2] + "), "
    return baseQuery

In [16]:
baseInsertQuery="INSERT INTO " + database + "." + schema + "." + table_name + "(" + (", ".join(columnsList)) +") VALUES " 


In [17]:

count=0
while count<merged_data.shape[0]:
    insertQuery=formQuery(baseInsertQuery,merged_data.iloc[count:count+15000,], merged_data.columns)

    ctx.cursor().execute(insertQuery[:-2])
    
    count+=15000
insertQuery=formQuery(baseInsertQuery,merged_data.iloc[count-15000:,],merged_data.columns)
ctx.cursor().execute(insertQuery[:-2])


In [18]:
copyTable="CREATE OR REPLACE TABLE " + database + "." + schema + "." + table_name +""" as """ +"SELECT DISTINCT * FROM " + database + "." + schema + "." + table_name
ctx.cursor().execute(copyTable)